## File API Examples

This notebook demonstrates how to work with Evo Files using **direct API calls**, providing granular control over file operations and detailed insight into the underlying service interactions.

### API vs SDK Approach

This notebook uses **direct File API calls** which:
- ✅ Provides full control over API requests and responses
- ✅ Shows detailed API response structures and metadata
- ✅ Allows for advanced customization and fine-tuning
- ✅ Helps understand the underlying service architecture
- ✅ Useful for debugging and advanced use cases

### Want a Simpler, High-Level Interface?

If you prefer a more streamlined experience with less boilerplate code, check out the `sdk-examples.ipynb` notebook in this same directory. The SDK examples show:
- Simplified method calls using `FileAPIClient`
- Automatic handling of complex API interactions
- Better error handling and validation
- Recommended for most common use cases

In [ ]:
from evo.notebooks import ServiceManagerWidget

# Evo app credentials
client_id = "<your-client-id>"  # Replace with your client ID
redirect_url = "<your-redirect-url>"  # Replace with your redirect URL

manager = await ServiceManagerWidget.with_auth_code(
    redirect_url=redirect_url,
    client_id=client_id,
).login()

### Prepare Evo SDK parameters

In [ ]:
# Get the environment and connector from the ServiceManagerWidget instance.
# The environment contains the hub URL, organization ID, and workspace ID.
# The connector is used to make API calls to the Evo service.
environment = manager.get_environment()
connector = manager.get_connector()

# Copy the environment details to local variables for easier access.
evo_hub_url = environment.hub_url
org_id = environment.org_id
workspace_id = environment.workspace_id

## File operations

### List all files in the workspace.

In [ ]:
import json
from http import HTTPStatus

import pandas as pd
from pygments import highlight
from pygments.formatters import TerminalTrueColorFormatter
from pygments.lexers import JsonLexer

from evo.common.data import HTTPResponse

try:
    resource_path = "/file/v2/orgs/{organisation_id}/workspaces/{workspace_id}/files"

    path_params = {
        "organisation_id": org_id,
        "workspace_id": workspace_id,
    }

    api_response = await connector.call_api(
        method="GET",
        resource_path=resource_path,
        path_params=path_params,
        response_types_map={
            "200": HTTPResponse,
        },
    )

    status = api_response.status
    if status != HTTPStatus.OK:
        raise RuntimeError(f"Error: Failed to list files. Status: {status}")

    # Parse the response data
    response = api_response.data.decode("utf-8")
    response_json = json.loads(response)

    # Print the response in a highlighted format
    print(highlight(json.dumps(response_json, indent=4), JsonLexer(), TerminalTrueColorFormatter()))

    # Display as a table
    file_list = [
        {
            "Name": file["name"],
            "Path": file["path"] + file["name"],
            "ID": file["file_id"],
            "Version_id": file["version_id"],
        }
        for file in response_json["files"]
    ]

    df = pd.DataFrame(file_list)
    display(df)
except Exception as e:
    print(f"Error listing files:\n{e}")

### List paginated files

If you have access to more than 50 files, you can use pagination to efficiently iterate through your files.

In [ ]:
import json
from http import HTTPStatus

import pandas as pd
from pygments import highlight
from pygments.formatters import TerminalTrueColorFormatter
from pygments.lexers import JsonLexer

from evo.common.data import HTTPResponse

try:
    resource_path = "/file/v2/orgs/{organisation_id}/workspaces/{workspace_id}/files"

    path_params = {
        "organisation_id": org_id,
        "workspace_id": workspace_id,
    }

    query_params = {
        "limit": 5,
        "offset": 0,
    }

    api_response = await connector.call_api(
        method="GET",
        resource_path=resource_path,
        path_params=path_params,
        query_params=query_params,
        response_types_map={
            "200": HTTPResponse,
        },
    )

    status = api_response.status
    if status != HTTPStatus.OK:
        raise RuntimeError(f"Error: Failed to list files. Status: {status}")

    # Parse the response data
    response = api_response.data.decode("utf-8")
    response_json = json.loads(response)

    # Print the response in a highlighted format
    print(highlight(json.dumps(response_json, indent=4), JsonLexer(), TerminalTrueColorFormatter()))

    # Display as a table
    file_list = [
        {
            "Name": file["name"],
            "Path": file["path"] + file["name"],
            "ID": file["file_id"],
            "Version_id": file["version_id"],
        }
        for file in response_json["files"]
    ]

    df = pd.DataFrame(file_list)
    display(df)
except Exception as e:
    print(f"Error listing files:\n{e}")

### Get blob storage URL

In [ ]:
import json
from http import HTTPStatus

import pandas as pd
from pygments import highlight
from pygments.formatters import TerminalTrueColorFormatter
from pygments.lexers import JsonLexer

from evo.common.data import HTTPResponse

file_path = "sample-data/example.txt"

try:
    resource_path = "/file/v2/orgs/{org_id}/workspaces/{workspace_id}/files/path/{file_path}"

    path_params = {
        "org_id": org_id,
        "workspace_id": workspace_id,
        "file_path": file_path,
    }

    api_response = await connector.call_api(
        method="PUT",
        resource_path=resource_path,
        path_params=path_params,
        header_params={"Content-Type": "application/json"},
        response_types_map={
            "200": HTTPResponse,
        },
    )

    status = api_response.status
    if status != HTTPStatus.OK:
        raise RuntimeError(f"Error: Failed to get a blob storage URL. Status: {status}")

    # Parse the response data
    response = api_response.data.decode("utf-8")
    response_json = json.loads(response)

    file_id = response_json["file_id"]

    # Print the response in a highlighted format
    print(highlight(json.dumps(response_json, indent=4), JsonLexer(), TerminalTrueColorFormatter()))

except Exception as e:
    print(f"Error getting blob storage URL:\n{e}")

### Upload the blob data

In [ ]:
import requests

file_contents = {f"{file_id}": open(file_path, "rb")}
upload_response = requests.put(
    response_json["upload"],
    files=file_contents,
    headers={"x-ms-blob-type": "BlockBlob"},
)

status = api_response.status
if status != HTTPStatus.OK:
    raise RuntimeError(f"Error: Failed to upload the file. Status: {status}")
else:
    print(f"Successfully uploaded file with ID: {file_id}")

## Poll until the file is ready

This example gives an example of looping while we wait for the file upload to complete in the background.

**Tip:** if you're uploading a new version of an existing file, you can compare the `version_id` you receive in the first and last steps of this example to determine whether the new version has finished uploading. 

In [ ]:
import json
import time
from http import HTTPStatus

import pandas as pd
from pygments import highlight
from pygments.formatters import TerminalTrueColorFormatter
from pygments.lexers import JsonLexer

from evo.common.data import HTTPResponse

i = 1
limit = 10  # To prevent an infinite loop

while True:
    resource_path = "/file/v2/orgs/{org_id}/workspaces/{workspace_id}/files/path/{file_path}"

    path_params = {
        "org_id": org_id,
        "workspace_id": workspace_id,
        "file_path": file_path,
    }

    api_response = await connector.call_api(
        method="GET",
        resource_path=resource_path,
        path_params=path_params,
        header_params={"Content-Type": "application/json"},
        response_types_map={
            "200": HTTPResponse,
        },
    )

    status = api_response.status
    if status != HTTPStatus.OK:
        raise RuntimeError(f"Error: Failed to upload the file. Status: {status}")

    # Parse the response data
    response = api_response.data.decode("utf-8")
    response_json = json.loads(response)

    # Is the file ready yet?
    if response_json:
        break
    # Have we reached our recursion limit?
    i += 1
    if i > limit:
        break
    # Wait a bit until trying again
    time.sleep(1)

print(highlight(json.dumps(response_json, indent=4), JsonLexer(), TerminalTrueColorFormatter()))

### Download a file

The `upload` URL returned by this request is a pre-signed download URL. It is valid for 30 minutes.

Use the `upload` URL to either open a new tab in your browser, perform a GET request to retrieve the file's contents, etc. 

If you need to download a specific version of a file, you can include `version={version_id}` in your request parameters.

In [ ]:
from http import HTTPStatus

from evo.common.data import HTTPResponse

try:
    resource_path = "/file/v2/orgs/{org_id}/workspaces/{workspace_id}/files/path/{file_path}"

    path_params = {
        "org_id": org_id,
        "workspace_id": workspace_id,
        "file_path": file_path,
    }

    api_response = await connector.call_api(
        method="GET",
        resource_path=resource_path,
        path_params=path_params,
        header_params={"Content-Type": "application/json"},
        response_types_map={
            "200": HTTPResponse,
        },
    )

    status = api_response.status
    if status != HTTPStatus.OK:
        raise RuntimeError(f"Error: Failed to get a download URL. Status: {status}")

    # Parse the response data
    response = api_response.data.decode("utf-8")
    download_url = json.loads(response)["download"]

    display(download_url)

except Exception as e:
    print(f"Error getting a download URL:\n{e}")

### Delete a file

When you delete a file, it will be soft-deleted. Deleted files will no longer be accessible through list files by default.

In [ ]:
from http import HTTPStatus

from evo.common.data import HTTPResponse

try:
    resource_path = "/file/v2/orgs/{org_id}/workspaces/{workspace_id}/files/{file_id}"

    path_params = {
        "org_id": org_id,
        "workspace_id": workspace_id,
        "file_id": file_id,
    }

    api_response = await connector.call_api(
        method="DELETE",
        resource_path=resource_path,
        path_params=path_params,
        response_types_map={
            "204": HTTPResponse,
        },
    )

    status = api_response.status
    if status != HTTPStatus.NO_CONTENT:
        raise RuntimeError(f"Error: Failed to delete file. Status: {status}")

    print(f"Successfully deleted file with ID: {file_id}")
except Exception as e:
    print(f"Error deleting file:\n{e}")

### Fetch details of a deleted file

You must include the `deleted=True` parameter to find details of a deleted file.

In [ ]:
import json
from http import HTTPStatus

import pandas as pd
from pygments import highlight
from pygments.formatters import TerminalTrueColorFormatter
from pygments.lexers import JsonLexer

from evo.common.data import HTTPResponse

try:
    resource_path = "/file/v2/orgs/{org_id}/workspaces/{workspace_id}/files/{file_id}"

    path_params = {
        "org_id": org_id,
        "workspace_id": workspace_id,
        "file_id": file_id,
    }

    query_params = {
        "deleted": "true",
    }

    api_response = await connector.call_api(
        method="GET",
        resource_path=resource_path,
        path_params=path_params,
        query_params=query_params,
        response_types_map={
            "200": HTTPResponse,
        },
    )

    status = api_response.status
    if status != HTTPStatus.OK:
        raise RuntimeError(f"Error: Failed to get deleted file. Status: {status}")

    # Parse the response data
    response = api_response.data.decode("utf-8")
    response_json = json.loads(response)

    # Print the response in a highlighted format
    print(highlight(json.dumps(response_json, indent=4), JsonLexer(), TerminalTrueColorFormatter()))

    # Display as a table
    file_list = [
        {
            "Name": response_json["name"],
            "Path": response_json["path"] + response_json["name"],
            "ID": response_json["file_id"],
            "Version_id": response_json["version_id"],
        }
    ]

    df = pd.DataFrame(file_list)
    display(df)
except Exception as e:
    print(f"Error fetching deleted file:\n{e}")

### Restore a deleted file

In [ ]:
from http import HTTPStatus

from evo.common.data import HTTPResponse

try:
    resource_path = "/file/v2/orgs/{org_id}/workspaces/{workspace_id}/files/{file_id}"

    path_params = {
        "org_id": org_id,
        "workspace_id": workspace_id,
        "file_id": response_json["file_id"],
    }

    query_params = {
        "deleted": "false",
    }

    api_response = await connector.call_api(
        method="PUT",
        resource_path=resource_path,
        path_params=path_params,
        header_params={"Content-Type": "application/json"},
        query_params=query_params,
        response_types_map={
            "204": HTTPResponse,
        },
    )

    status = api_response.status
    if status != HTTPStatus.NO_CONTENT:
        raise RuntimeError(f"Error: Failed to restore file. Status: {status}")
    else:
        print(f"Successfully restored file with ID: {response_json['file_id']}")

except Exception as e:
    print(f"Error restoring file:\n{e}")